<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/AIGCDetectBenchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIGCDetectBenchmark


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install ftfy
!pip install natsort
!pip install tensorboardX
!pip install blobfile
!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 864.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746516 sha256=b558542ede113a9c5f96754a9b5b37620cc9a2437aebf89753c6fc1b32be8b64
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [3]:
!git clone https://github.com/vohoaidanh/AIGCDetectBenchmark.git

Cloning into 'AIGCDetectBenchmark'...
remote: Enumerating objects: 571, done.
remote: Counting objects: 100% (571/571), done.
remote: Compressing objects: 100% (399/399), done.
remote: Total 571 (delta 236), reused 463 (delta 155), pack-reused 0
Receiving objects: 100% (571/571), 6.46 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [ ]:
 #file_path = '/content/drive/MyDrive/DATASETS/PIE_Net_real_gen_dataset.zip'
 #!mkdir -p /content/AIGCDetectBenchmark/dataset
 #!unzip "$file_path" -d /content/AIGCDetectBenchmark/dataset

In [ ]:
#!cp '/content/drive/MyDrive/WEIGHTS/sidd_rgb.pth' /content/AIGCDetectBenchmark/weights/preprocessing

In [4]:
import os
import shutil

def copy_images(source_folder, destination_folder):
    # Check if the source folder exists
    if not os.path.exists(source_folder):
        print("Source folder does not exist.")
        return

    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Recursively copy the entire directory tree
    for root, dirs, files in os.walk(source_folder):
        for filename in files:
            # Check if the file is an image (you can customize this condition)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                source_path = os.path.join(root, filename)
                relative_path = os.path.relpath(source_path, source_folder)
                destination_path = os.path.join(destination_folder, relative_path)
                destination_dir = os.path.dirname(destination_path)
                # Create the directory if it doesn't exist
                if not os.path.exists(destination_dir):
                    os.makedirs(destination_dir)
                # Copy the image file to the destination folder
                shutil.copyfile(source_path, destination_path)
                print(f"Copied {filename} to {destination_folder}")

# Example usage:
#source_folder = '/path/to/source_folder'
#destination_folder = '/path/to/destination_folder'
#copy_images(source_folder, destination_folder)


In [5]:
import os
import shutil

def move_files_(source_folder, destination_folder, indices):
    # Check if the source folder exists
    if not os.path.exists(source_folder):
        print("Source folder does not exist.")
        return

    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Move files specified by indices from source folder to destination folder
    for idx in indices:
        filename = os.listdir(source_folder)[idx]
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.move(source_path, destination_path)
        print(f"Moved {filename} to {destination_folder}")

# Example usage:
#source_folder = '/path/to/source_folder'
#destination_folder = '/path/to/destination_folder'
#copy_images(source_folder, destination_folder)


In [6]:
source_folder = '/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet'
destination_folder = '/content/AIGCDetectBenchmark/dataset'
copy_images(source_folder, destination_folder)

Streaming output truncated to the last 5000 lines.
Copied 22_albedo_1833.png to /content/AIGCDetectBenchmark/dataset
Copied 22_shading_1833.png to /content/AIGCDetectBenchmark/dataset
Copied 22_origin_1833.png to /content/AIGCDetectBenchmark/dataset
Copied 11_albedo_1834.png to /content/AIGCDetectBenchmark/dataset
Copied 11_shading_1834.png to /content/AIGCDetectBenchmark/dataset
Copied 11_origin_1834.png to /content/AIGCDetectBenchmark/dataset
Copied 19_albedo_1835.png to /content/AIGCDetectBenchmark/dataset
Copied 19_shading_1835.png to /content/AIGCDetectBenchmark/dataset
Copied 19_origin_1835.png to /content/AIGCDetectBenchmark/dataset
Copied 31_albedo_1836.png to /content/AIGCDetectBenchmark/dataset
Copied 31_shading_1836.png to /content/AIGCDetectBenchmark/dataset
Copied 31_origin_1836.png to /content/AIGCDetectBenchmark/dataset
Copied 1_albedo_1837.png to /content/AIGCDetectBenchmark/dataset
Copied 1_shading_1837.png to /content/AIGCDetectBenchmark/dataset
Copied 1_origin_1837.p

In [37]:
import os
source = '/content/AIGCDetectBenchmark/dataset/train/0_real'
dest = '/content/AIGCDetectBenchmark/dataset/val/0_real'

origin_list = os.listdir(source)
origin_list = [i for i in origin_list if 'origin' in i]

In [38]:
import numpy as np
l = len(origin_list)
train_size = (int(0.8 * l))
train_list = np.random.choice(origin_list, train_size,replace=False)
val_list = list(set(origin_list) - set(train_list))


In [8]:
!mkdir /content/AIGCDetectBenchmark/dataset/val/0_real
!mkdir /content/AIGCDetectBenchmark/dataset/val/1_fake

384

In [39]:
from tqdm import tqdm
for f in tqdm(val_list):
  source_path = os.path.join(source, f)
  source_path2 = source_path.replace('origin', 'shading')
  source_path3 = source_path.replace('origin', 'albedo')

  destination_path = os.path.join(dest, f)
  destination_path2 = destination_path.replace('origin', 'shading')
  destination_path3 = destination_path.replace('origin', 'albedo')


  shutil.move(source_path, destination_path)
  shutil.move(source_path2, destination_path2)
  shutil.move(source_path3, destination_path3)

  print(f"Moved {source_path} to {destination_path}")

  9%|▉         | 96/1011 [00:00<00:00, 955.38it/s]

Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00033163_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00033163_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00037838_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00037838_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00013656_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00013656_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00017496_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00017496_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00037797_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00037797_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00025491_origin_.png to /content/A

 43%|████▎     | 436/1011 [00:00<00:00, 1500.45it/s]

Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00027996_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00027996_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00016666_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00016666_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00001931_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00001931_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00041195_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00041195_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00040998_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00040998_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00010675_origin_.png to /content/A

100%|██████████| 1011/1011 [00:00<00:00, 1843.61it/s]

Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00013629_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00013629_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00037170_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00037170_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00024386_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00024386_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00013158_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00013158_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00017363_origin_.png to /content/AIGCDetectBenchmark/dataset/val/0_real/ILSVRC2012_val_00017363_origin_.png
Moved /content/AIGCDetectBenchmark/dataset/train/0_real/ILSVRC2012_val_00021256_origin_.png to /content/A

In [40]:
%cd /content/AIGCDetectBenchmark

/content/AIGCDetectBenchmark


In [ ]:
#/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train
#import shutil

#shutil.copytree('/content/AIGCDetectBenchmark/dataset/train', '/content/AIGCDetectBenchmark/dataset/val')
#!rm -r /content/AIGCDetectBenchmark/dataset/content

'/content/AIGCDetectBenchmark/dataset/val'

In [ ]:
#!python test_sidd_rgb_test.py --input_dir dataset/real_gen_dataset --result_dir dataset/data

In [41]:
!python train.py --name test_with_intrinsic --dataroot /content/AIGCDetectBenchmark/dataset \
 --detect_method intrinsic --blur_prob 0.1 --blur_sig 0.0,3.0 --jpg_prob 0.1 --jpg_method cv2,pil --jpg_qual 30,100

----------------- Options ---------------
                 CropSize: 224                           
               batch_size: 64                            
                    beta1: 0.9                           
                blur_prob: 0.1                           
                 blur_sig: 0.0,3.0                       
          checkpoints_dir: ./checkpoints                 
                  classes: ['airplane', 'bird', 'bicycle', 'boat', 'bottle', 'bus', 'car', 'cat', 'cow', 'chair', 'diningtable', 'dog', 'person', 'pottedplant', 'motorbike', 'tvmonitor', 'train', 'sheep', 'sofa', 'horse']	[default: airplane,bird,bicycle,boat,bottle,bus,car,cat,cow,chair,diningtable,dog,person,pottedplant,motorbike,tvmonitor,train,sheep,sofa,horse]
           continue_train: False                         
                 data_aug: False                         
                 dataroot: /content/AIGCDetectBenchmark/dataset	[default: /hotdata/share/AIGCDetect]
            detect_method:

In [42]:
!cp /content/AIGCDetectBenchmark/checkpoints/test_with_intrinsic/model_epoch_best.pth /content/drive/MyDrive/WEIGHTS/intrinsic_method

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Đường dẫn đến event file được tạo bởi SummaryWriter
event_file_path = 'checkpoints/test_with_PIE_Net/val/events.out.tfevents.1709110494.4cc1337ca71b'

# Đọc dữ liệu từ event file
summary_iterator = tf.compat.v1.train.summary_iterator(event_file_path)

# Khởi tạo các list để lưu giữ dữ liệu
steps = []
train_losses = []
train_accuracies = []

# Lặp qua từng event trong event file
for event in summary_iterator:
    if event.HasField('summary'):
        for value in event.summary.value:
            if value.tag == 'accuracy':
                steps.append(event.step)
                train_losses.append(value.simple_value)
            elif value.tag == 'ap':
                train_accuracies.append(value.simple_value)

# Vẽ đồ thị train loss
plt.plot(steps, train_losses, label='accuracy')
plt.xlabel('Steps')
plt.ylabel('accuracy')
plt.title('Validate accuracy over Steps')
plt.legend()
plt.show()

# Vẽ đồ thị train accuracy
plt.plot(steps, train_accuracies, label='Val AP')
plt.xlabel('Steps')
plt.ylabel('AP')
plt.title('Validate AP over Steps')
plt.legend()
plt.show()


896

In [ ]:
!python eval_all.py

RealFakeDB_tiny
----------------- Options ---------------
                 CropSize: 224                           
           DIRE_modelpath: /data_19/AIGCDetect/AIGCDetect/weights/preprocessing/lsun_bedroom.pt
          LGrad_modelpath: ./weights/preprocessing/karras2019stylegan-bedrooms-256x256_discriminator.pth
            LNP_modelpath: weights/preprocessing/sidd_rgb.pth
               batch_size: 64                            
                 blur_sig: 1.0                           
            detect_method: CNNSpot                       
                  isTrain: False                         	[default: None]
                    isVal: False                         	[default: None]
               jpg_method: pil                           
                 jpg_qual: 95                            
                 loadSize: 256                           
               model_path: checkpoints/test_with_PIE_Net/model_epoch_best.pth
                  no_crop: False               

In [ ]:
#!cp /content/drive/MyDrive/DATASETS/real_gen_dataset.zip /content/

In [ ]:
#!unzip /content/drive/MyDrive/DATASETS/RealFakeDB_tiny.zip -d /content/AIGCDetectBenchmark

In [ ]:
#!cp /content/AIGCDetectBenchmark/checkpoints/test_with_PIE_Net/model_epoch_best.pth -d /content/drive/MyDrive/WEIGHTS/PIE_Net_and_CNNSpot